In [1]:
# Resources: 
# Code lists, definitions, accuracy: https://www.census.gov/programs-surveys/acs/technical-documentation/code-lists.html
# Gitter help: https://gitter.im/uscensusbureau/general
# Variables: https://api.census.gov/data/2017/acs/acs5/variables.html
# available datasets: https://api.census.gov/data.html
# available geographies: https://api.census.gov/data/2009/acs5/geography.html
# instructional PPT: https://www.census.gov/content/dam/Census/programs-surveys/acs/guidance/training-presentations/06212017_ACS_Census_API.pdf

In [33]:
import requests
import urllib.request
import time
import pandas as pd
import json
import censusdata
import math
import os
from pathlib import Path

n_drive = 'N:\Transfer\KSkvoretz\AHRQ\data\\01_Demographic\ACS'
output = os.path.join(Path(n_drive).parents[0], 'cleaned')

key = 'b34f5dfe18f660a15a278a309760c38ef401b395'

N:\Transfer\KSkvoretz\AHRQ\data\01_Demographic\cleaned


# Read in table shells where I've flagged which variables to use

In [3]:
table_shell = os.path.join(n_drive, 'ACS2017_Table_Shells.xlsx')
xl = pd.ExcelFile(table_shell)

table_shell_df = xl.parse(xl.sheet_names[0])
# variables I've flagged to use
use_vars = table_shell_df[table_shell_df.Use == 1]
print(use_vars)
print(use_vars[['TableID','Stub','Use']])

# Potentially others?
# disability
# vision/hearing difficulty?
# other stuff by race?? -i.e. health insurance?
# internet
# population coverage

# add category to variable names?

use_vars.to_csv(os.path.join(n_drive, 'ACS_variables.csv'))

      TableID Line UniqueID  \
9      B01001           NaN   
549    B02001  NaN      NaN   
1463   B05001           NaN   
2955   B06007           NaN   
3097   B06008           NaN   
...       ...  ...      ...   
31509  B25100           NaN   
34862  B27001  NaN      NaN   
34922  C27001  NaN      NaN   
36014  B27010  NaN      NaN   
36083  C27010  NaN      NaN   

                                                    Stub Data Release  Use  
9                                             SEX BY AGE         1,5   1.0  
549                                                 RACE         1,5   1.0  
1463   NATIVITY AND CITIZENSHIP STATUS IN THE UNITED ...         1,5   1.0  
2955   PLACE OF BIRTH BY LANGUAGE SPOKEN AT HOME AND ...         1,5   1.0  
3097   PLACE OF BIRTH BY MARITAL STATUS IN THE UNITED...         1,5   1.0  
...                                                  ...          ...  ...  
31509  MORTGAGE STATUS BY RATIO OF VALUE TO HOUSEHOLD...         1,5   1.0  
34862     H

In [4]:
variables = use_vars.TableID.tolist()

# Method 1: Scrape data from API

In [5]:
def api_pull(variable):
    """
    
    if get an error here, use jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
    """
    # all the B01001 variables across US or by state for 5 year ACS
    # acs5 = dataset name
    # ?key = 
    # get = variable,names,separated,by,commas
    # &for=geography level:code or * for all

    # i.e. &for=county+subdivision:*&in=state:36, &for=county+subdivision:*&in=state:36+county:001

    url = f'https://api.census.gov/data/2017/acs/acs5?key={key}&get=NAME,group({variables[0]})&for=county:*&in=state:08'

    response = requests.get(url)

    formattedResponse = json.loads(response.text)[1:]

    data = pd.DataFrame(formattedResponse)
    return data

In [6]:
# what are these columns though?
co = api_pull(variables[0])
print(co.head())
print(co.shape)

                         0               1       2           3       4    5    \
0  Costilla County, Colorado  0500000US08023    3628  -555555555    1958   41   
1     Adams County, Colorado  0500000US08001  487850  -555555555  245840  136   
2   Conejos County, Colorado  0500000US08021    8147  -555555555    4084   28   
3    Custer County, Colorado  0500000US08027    4505  -555555555    2200  116   
4   Douglas County, Colorado  0500000US08035  320940  -555555555  159804   71   

     6    7      8    9    ...   191   192   193   194   195   196   197  \
0    101   33    120   61  ...  None  None  None  None  None  None  None   
1  18737  119  19722  861  ...  None  None  None  None  None  None  None   
2    302   27    260   44  ...  None  None  None  None  None  None  None   
3     45   35     48   42  ...  None  None  None  None  None  None  None   
4  10380  116  12841  551  ...  None  None  None  None  None  None  None   

    198 199  200  
0  None  08  023  
1  None  08  001  

# Method 2: Use the census data package

In [7]:
# Explore functionality
censusdata.search('acs5', 2017, 'label','unemploy')
# censusdata.search('acs5', 2017, 'concept', 'education')
censusdata.printtable(censusdata.censustable('acs5',2017, 'B23025'))
censusdata.geographies(censusdata.censusgeo([('state','*')]), 'acs5', 2017)
censusdata.geographies(censusdata.censusgeo([('state', '08'), ('county', '*')]), 'acs5', 2017)

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B23025_001E  | EMPLOYMENT STATUS FOR THE POPU | !! Estimate Total                                        | int  
B23025_002E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total In labor force                      | int  
B23025_003E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Civilian labor fo | int  
B23025_004E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_005E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_006E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Armed Forces      | int  
B23025_007E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total Not in labor force      

{'Costilla County, Colorado': censusgeo((('state', '08'), ('county', '023'))),
 'Adams County, Colorado': censusgeo((('state', '08'), ('county', '001'))),
 'Conejos County, Colorado': censusgeo((('state', '08'), ('county', '021'))),
 'Custer County, Colorado': censusgeo((('state', '08'), ('county', '027'))),
 'Douglas County, Colorado': censusgeo((('state', '08'), ('county', '035'))),
 'Eagle County, Colorado': censusgeo((('state', '08'), ('county', '037'))),
 'Lake County, Colorado': censusgeo((('state', '08'), ('county', '065'))),
 'La Plata County, Colorado': censusgeo((('state', '08'), ('county', '067'))),
 'Moffat County, Colorado': censusgeo((('state', '08'), ('county', '081'))),
 'Pitkin County, Colorado': censusgeo((('state', '08'), ('county', '097'))),
 'Rio Grande County, Colorado': censusgeo((('state', '08'), ('county', '105'))),
 'Cheyenne County, Colorado': censusgeo((('state', '08'), ('county', '017'))),
 'Bent County, Colorado': censusgeo((('state', '08'), ('county', '01

In [8]:
def censusdata_pull(variable, acs_version = 'acs5', year = 2017, max_vars = 49.0):
    """
    Used to pull and merge data for multiple variables, since we are using much more than 50
    and 50 is the limit to pull from the API (According to the limit, it's 50, but it's actually 49)
    
    Args:
        variable (string): variable group name
        acs_version (string): 'acs5' for the 5-year survey
        year (int): 2017 is the most recent as of now
        max_vars (float): If API limit ever changes, we can adjust this default value from 49.0
        
    Returns:
        dataframe
    
    """
    
    # TODO: create a test to count all variables and match with number of columns in final dataframe
    # TODO: create a dictionary of variable names with column names
    
    # Create a list of all variable names found related to the input variable group
    census_dict = censusdata.censustable(acs_version,year,variable)
    unique_ids = list(census_dict.keys())
    
    # The API sets a limit of pulling 50 variables at a time
    num_vars = len(unique_ids)    
    # Number of loops we'll have to do to pull groups of 50 or less variables
    num_loops = math.ceil(num_vars/max_vars)
    
#     print(num_vars)
#     print(num_loops)
    
    # used to store the indices of the 50 variables to be pulled
    last = int(max_vars)
    first = 0
    for i in range(num_loops):
        
        print(len(unique_ids[first:last]))
        data = censusdata.download(acs_version, year,
                              censusdata.censusgeo([('state', '08'), ('county', '*')]),
                              unique_ids[first:last])
        
        # rename columns from variable names
        new_colnames = {}
        for key, value in census_dict.items():
            new_name = value['concept'] + "_" + value['label']
            new_name = new_name.replace('!!', "_").replace(" ", "_")
            new_colnames[key] = new_name

        data.rename(columns = new_colnames, inplace = True)
        
        if i == 0:
            full_data = data
        else:
            # merge the data by county
            full_data = full_data.join(data, how = 'outer')
        
        # increment to 50 variables later
        last += int(max_vars)
        first = last-int(max_vars)
        if last > num_vars:
            last = num_vars
    
    return full_data

In [9]:
# doesn't seem like the C variables work, so remove them
variables = [var for var in variables if 'C' not in var]
variables = [var for var in variables if "B17002" not in var]

In [10]:
%time
# loop through all variables and merge data together
count = 0
for variable in variables:
    print(variable)
    data = censusdata_pull(variable)
    
    if count == 0:
        full_data = data
    else:
        full_data = full_data.join(data, how = 'outer')
    count += 1

Wall time: 0 ns
B01001
49
B02001
10
B05001
6
B06007
40
B06008
30
B06010
49
6
B06012
20
B07001
49
47
B08006
49
2
B08012
39
B08126
49
49
7
B08128
49
21
B08202
22
B08301
21
B08303
13
B08524
49
B09001
10
B09002
20
B09005
6
B09008
12
B09010
13
B09018
8
B09021
28
B11001
9
B11016
16
B12001
19
B12006
49
7
B12007
2


C:\Users\kskvoretz\Documents\Projects\.virtualenvs\AHRQ-ebGvUS5S\lib\site-packages\pandas\core\indexes\base.py:3597: RuntimeWarning: '<' not supported between instances of 'censusgeo' and 'censusgeo', sort order is undefined for incomparable objects
  join_index = self.union(other)


B12505
13
B15001
49
34
B17008
3
B19001
17
B19013
1
B19025
1
B19057
3
B19058
3
B19081
6
B19083
1
B19301
1
B19301A
1
B19301B
1
B21001
39
B23018
3
B23020
3
B24081
9
B25001
1
B25002
3
B25003
3
B25031
7
B25041
7
B25070
11
B25071
1
B25075
27
B25077
1
B25094
18
B25096
21
B25097
3
B25100
13
B27001
49
8
B27010
49
17


In [12]:
print(full_data.shape)

(64, 1302)


In [34]:
print(output)
full_data.to_csv(os.path.join(output, 'ACS_cleaned.csv'))

N:\Transfer\KSkvoretz\AHRQ\data\01_Demographic\cleaned
